In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key = os.getenv('GROQ_API_KEY')
from langchain_groq import ChatGroq

model = ChatGroq(model='Llama3-8B-8192',api_key=groq_api_key)

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
from langchain_huggingface import HuggingFaceEmbeddings
embeddings= HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/home/priteshs/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain.chains import create_retrieval_chain  #  Create retrieval chain that retrieves documents and then passes them on.
from langchain.chains.combine_documents import create_stuff_documents_chain  # Create a chain for passing a list of Documents to a model.
import bs4

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
loader = WebBaseLoader(
    web_paths = ('https://lilianweng.github.io/posts/2023-06-23-agent/',),
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            class_=('post-content', 'post-title', 'post-header')
        )
    )
)

In [22]:
web_content = loader.load()
documents = splitter.split_documents(web_content)
vectorstore = Chroma.from_documents(documents, embedding=embeddings)
retriver = vectorstore.as_retriever()

retriver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x70c5d1bbecf0>, search_kwargs={})

In [30]:
system_prompt = (
    'you are an assistant for question-answering tasks.'
    'use the following piece of retrieved context to answer'
    'the question. if you dont know the answer, say that you'
    'dont know. use three sentence maximum and keep the'
    'answer concise'
    '\n\n'
    '{context}'
)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('user','{input}')
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [31]:
res = rag_chain.invoke({'input':'what is self-reflection'})

In [33]:
res['answer']

'Self-reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

In [56]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    'Given a chat history and the latest user experience'
    'which might reference context in the chat history'
    'formulate a standalone question which can be understood'
    'without the chat history. Do not answer the question,'
    'just reformulate it if needed and otherwise return it as it'
)
 
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',contextualize_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('user','{input}')
    ]
)

In [41]:
history_aware_retriever = create_history_aware_retriever(model, retriver, contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x70c5d1bbecf0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChu

In [42]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [43]:
question_answer_chain=create_stuff_documents_chain(model,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [44]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x70c5d1bbecf0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annota

In [47]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Self-Reflection"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question2,"chat_history":chat_history})
print(response2['answer'])

I don't know. The provided context only provides a brief description of Self-Reflection and its importance in autonomous agents. If you're looking for more information, I would need additional context or a more specific question about Self-Reflection.


In [49]:
chat_history

[HumanMessage(content='What is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.', additional_kwargs={}, response_metadata={})]

In [52]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory 

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer'
)

In [53]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts (Yao et al. 2023) to decompose the task into multiple thought steps, creating a step-by-step plan for achieving the desired outcome.'

In [54]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Common ways of doing Task Decomposition include: (1) using Large Language Models (LLM) with simple prompting, (2) using task-specific instructions, or (3) with human inputs.'

In [60]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts (Yao et al. 2023) to decompose the task into multiple thought steps, creating a step-by-step plan for achieving the desired outcome.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Common ways of doing Task Decomposition include: (1) using Large Language Models (LLM) with simple prompting, (2) using task-specific instructions, or (3) with human inputs.', additional_kwargs={}, response_metadata={})])}